# AlexNet iFood2019 - Train All Models

This notebook trains all 4 AlexNet variants sequentially:
1. alexnet_baseline
2. alexnet_mod1 (with BatchNorm)
3. alexnet_mod2 (with Dropout)
4. alexnet_combined (BatchNorm + Dropout + improvements)

In [ ]:
# ============================================================
# Cell 1: Setup
# ============================================================

from google.colab import drive
drive.mount('/content/drive')

import os
import sys

PROJECT_PATH = '/content/drive/MyDrive/AlexNet_iFood2019'
REPO_PATH = '/content/alexnet-ifood2019'

if not os.path.exists(REPO_PATH):
    !git clone https://github.com/deftorch/alexnet-ifood2019.git {REPO_PATH}

os.chdir(REPO_PATH)
sys.path.insert(0, REPO_PATH)

# Create symlinks
!rm -rf data checkpoints evaluation_results analysis_results
!ln -s {PROJECT_PATH}/dataset data
!ln -s {PROJECT_PATH}/checkpoints checkpoints
!ln -s {PROJECT_PATH}/evaluation_results evaluation_results
!ln -s {PROJECT_PATH}/analysis_results analysis_results

print("✓ Setup complete")

In [ ]:
# ============================================================
# Cell 2: Install Dependencies
# ============================================================

!pip install -q torch torchvision pandas numpy pillow scikit-learn matplotlib seaborn tqdm wandb

import torch
print(f"PyTorch: {torch.__version__}")
print(f"CUDA: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

In [ ]:
# ============================================================
# Cell 3: Training Configuration
# ============================================================

MODELS = [
    'alexnet_baseline',
    'alexnet_mod1',
    'alexnet_mod2',
    'alexnet_combined'
]

# Training settings
NUM_EPOCHS = 50
BATCH_SIZE = 128
LR = 0.01
USE_WANDB = False  # Set True for WandB logging

print("Models to train:")
for i, m in enumerate(MODELS, 1):
    print(f"  {i}. {m}")
print(f"\nEpochs: {NUM_EPOCHS}, Batch Size: {BATCH_SIZE}, LR: {LR}")

In [ ]:
# ============================================================
# Cell 4: Train All Models Sequentially
# ============================================================

import time

training_times = {}

for model_name in MODELS:
    print("\n" + "=" * 70)
    print(f"TRAINING: {model_name}")
    print("=" * 70)
    
    start_time = time.time()
    
    wandb_flag = "--use_wandb" if USE_WANDB else ""
    
    !python src/train.py \
        --data_dir data \
        --model_name {model_name} \
        --num_epochs {NUM_EPOCHS} \
        --batch_size {BATCH_SIZE} \
        --lr {LR} \
        --save_dir checkpoints \
        {wandb_flag}
    
    elapsed = time.time() - start_time
    training_times[model_name] = elapsed
    
    print(f"\n✓ Completed {model_name} in {elapsed/60:.1f} minutes")

print("\n" + "=" * 70)
print("ALL MODELS TRAINED!")
print("=" * 70)
for model, t in training_times.items():
    print(f"  {model}: {t/60:.1f} min")

In [ ]:
# ============================================================
# Cell 5: Evaluate All Models
# ============================================================

print("\nEvaluating all models...\n")

for model_name in MODELS:
    print(f"Evaluating: {model_name}")
    
    !python src/evaluate.py \
        --data_dir data \
        --model_path checkpoints/{model_name}_best.pth \
        --model_name {model_name} \
        --split val \
        --output_dir evaluation_results
    
    print(f"✓ Evaluated: {model_name}\n")

In [ ]:
# ============================================================
# Cell 6: Run Comparative Analysis
# ============================================================

!python src/analysis.py \
    --checkpoint_dir checkpoints \
    --eval_dir evaluation_results \
    --output_dir analysis_results \
    --models alexnet_baseline alexnet_mod1 alexnet_mod2 alexnet_combined

print("\n✓ Comparative analysis complete!")

In [ ]:
# ============================================================
# Cell 7: Display Results Summary
# ============================================================

import json
import pandas as pd

results = []

for model_name in MODELS:
    row = {'Model': model_name}
    
    # Load history
    history_file = f'checkpoints/{model_name}_history.json'
    if os.path.exists(history_file):
        with open(history_file) as f:
            history = json.load(f)
        row['Best Val Acc'] = max(history['val_acc'])
        row['Best Epoch'] = history['val_acc'].index(max(history['val_acc'])) + 1
    
    # Load metrics
    metrics_file = f'evaluation_results/{model_name}_val_metrics.json'
    if os.path.exists(metrics_file):
        with open(metrics_file) as f:
            metrics = json.load(f)
        row['Test Acc'] = metrics.get('accuracy', 0)
        row['Top-5 Acc'] = metrics.get('top5_accuracy', 0)
        row['Macro F1'] = metrics.get('macro_f1', 0)
    
    results.append(row)

df = pd.DataFrame(results)

print("\n" + "=" * 80)
print("FINAL RESULTS")
print("=" * 80)
print(df.to_string(index=False))
print("=" * 80)

# Find best model
if 'Best Val Acc' in df.columns:
    best_idx = df['Best Val Acc'].idxmax()
    best_model = df.iloc[best_idx]['Model']
    print(f"\n🏆 Best Model: {best_model}")

In [ ]:
# ============================================================
# Cell 8: Display Analysis Plots
# ============================================================

from IPython.display import Image, display
import os

plots = [
    'training_curves_comparison.png',
    'metrics_comparison.png',
    'confusion_matrices.png'
]

for plot in plots:
    path = f'analysis_results/{plot}'
    if os.path.exists(path):
        print(f"\n{plot}:")
        display(Image(filename=path, width=800))
    else:
        print(f"⚠️  Not found: {plot}")

In [ ]:
# ============================================================
# Cell 9: Sync to Drive
# ============================================================

print("Syncing results to Google Drive...")

# Results are already synced via symlinks
# But let's verify

print(f"\nCheckpoints: {len(os.listdir('checkpoints'))} files")
print(f"Evaluation results: {len(os.listdir('evaluation_results'))} files")
print(f"Analysis results: {len(os.listdir('analysis_results'))} files")

print(f"\n✓ All results saved to: {PROJECT_PATH}")
print("\n🎉 Training pipeline complete!")